In [1]:
#needed after restructuring of directory
import os
os.chdir('C:\\Users\\Alex\\OneDrive\\Documents\\GitHub\\UFC_Prediction_2022')

#!/usr/bin/env python
# coding: utf-8

# In[ ]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

#turning csv files to json files
import csv
import json
 
# Function to convert a CSV to JSON
# Takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath, column):
     
    # create a dictionary
    data = {}
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:
             
            # primary key given by column variable
            key = rows[column]
            data[key] = rows
 
    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))


# Testing 

In [2]:
def get_next_fight_card():
    url = 'https://www.bestfightodds.com'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser") 
    mycards = soup.find_all("div", {"class": "table-div"})
    ufc_cards = [card for card in mycards if 'ufc' in card.find("a",href=lambda x: x and x.startswith('/events')).get_text().lower()]
    card = ufc_cards[0]
    card_date = card.find("span",{"class":"table-header-date"}).get_text()
    card_title = card.find("a",href=lambda x: x and x.startswith('/events')).get_text()
    #bookies = [bookie.get_text() for bookie in card.find_all("a",href=lambda x: x and x.startswith('/out'))]
    fighter_divs = card.find_all('span',{"class":"t-b-fcc"})
    fighters_list = [fighter.get_text() for fighter in fighter_divs]
    if len(fighters_list)%2 == 0 and all(fighters_list[i] == fighters_list[i+26] for i in range(len(fighters_list)//2)):
        print('Usual Scraping Structure Detected')
    else:
        print('Scraping Structure Has Changed... Check HTML at BestFightOdds.com')
    fights_list = [fighters_list[i:i+2] for i in range(0,len(fighters_list),2)]
    print(f'Upcoming card {card_title} on {card_date} has {len(fights_list)} fights')
    for fight in fights_list:
        print(fight)
    return card_date, card_title, fights_list


In [ ]:
card_date, card_title, fights_list = get_next_fight_card()

# Lets make sure we can add this data in a way that doesn't break the existing data regime

Recall how the program works:

On Github Actions once per week we run

```
name: Update Data
    run: |
        cd src
        python3 update_data_csvs_and_jsons.py
        python3 update_and_rebuild_model.py
```

update_data_csvs_and_jsons does the following:
- bring fight_hist.csv up to date (contains stats from each individual fight)
- bring fighter_stats.csv up to date (contains physical stats from each individual fighter)
- bring ufc_fights_crap.csv up to date (this is compiled from fight_hist and includes computed average stats for each fight one can use to predict fights)
- bring ufc_fights.csv up to date (cleaned version of ufc_fights_crap which can actually be used to predict fights)
- convert ufc_fights_crap to json as ufcfightscrap.json to use in website computations
- scrapes pictures of new fighers added to the roster

updata_and_rebuild_model does the following:
- uses new csv files from above to rebuild the prediction model and updates jsons containing model coefficients for website predictions
- populates vegas_odds.json with new fights and their vegas odds (we will rid of the odds feature for now)
- makes predictions for all of the fights in vegas_odds.csv for which it can
- fills in predictions to vegas_odds.csv and concatenates these predictions to prediction_history.json
- saves over prediction_history.json


In [4]:
vegas_odds=pd.read_json('src/models/buildingMLModel/data/external/vegas_odds.json')
prediction_history = pd.read_json('src/models/buildingMLModel/data/external/prediction_history.json')

In [ ]:
vegas_odds

In [5]:
prediction_history.head()

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref,predicted fighter odds,predicted opponent odds,average bookie odds,correct?
0,Angela Hill,,145,145,150,144,,+150,150,+150,146,,Mackenzie Dern,,-175,-170,-190,-186,,-190,-175,-188,-171,,305,-305,"[148, -181]",
1,Anthony Hernandez,,-210,-200,-235,-245,,-235,-225,-250,-191,,Edmen Shahbazyan,,170,170,185,186,,185,188,+200,166,,-139,139,"[-224, 181]",
2,Emily Ducote,,130,125,128,128,,+128,130,+120,133,,Loopy Godinez,,-160,-150,-159,-164,,-159,-160,-150,-153,,210,-210,"[128, -157]",
3,Andre Fialho,,185,190,180,172,,+180,188,+175,180,,Joaquin Buckley,,-225,-230,-230,-225,,-230,-225,-225,-210,,157,-157,"[181, -225]",
4,Diego Ferreira,,-155,-150,-167,-156,,-167,-160,-163,-163,,Michael Johnson,,125,125,135,122,,135,130,+130,143,,-144,144,"[-160, 131]",


Goad: Add new fights to prediction_history from `card_date, card_title, fights_list = get_next_fight_card()` instead of from vegas_odds.json

In [ ]:
col_names = list(prediction_history.columns)
col_values = [['' for _ in range(len(fights_list))] for _ in range(len(col_names))]
d = dict(zip(col_names, col_values))
vegas_odds = pd.DataFrame(data=d)

In [ ]:
import datetime
year = datetime.date.today().strftime('%B %d, %Y')[-4:]
year

In [ ]:
# input looks like 'July 15th'. Need to add year to it
def convert_scraped_date_to_standard_date(input_date):
    year = datetime.date.today().strftime('%B %d, %Y')[-4:]
    month = input_date.split(' ')[0]
    day = input_date.split(' ')[1]
    i = 0
    while day[i].isdigit():
        i+=1
    day = day[:i]
    return f'{month} {day}, {year}'

In [ ]:
convert_scraped_date_to_standard_date(card_date)

In [ ]:
vegas_odds['fighter name'].loc[:] = [fight[0] for fight in fights_list]
vegas_odds['opponent name'].loc[:] = [fight[1] for fight in fights_list]
vegas_odds['date'] = convert_scraped_date_to_standard_date(card_date)


In [ ]:
vegas_odds[['fighter name','opponent name','date']]

In [ ]:
#print('scraping bookie odds from bestfightodds.com')
#odds_df = get_odds()
#odds_df=drop_irrelevant_fights(odds_df,3) #allows 3 bookies to have missing odds. can increase this to 2 or 3 as needed
#odds_df=drop_non_ufc_fights(odds_df)
#odds_df=drop_repeats(odds_df)
#print('saving odds to models/buildingMLModel/data/external/vegas_odds.json')
#save to json
#result = odds_df.to_json()
#parsed = json.loads(result)
#jsonFilePath='models/buildingMLModel/data/external/vegas_odds.json'
#with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
#    jsonf.write(json.dumps(parsed, indent=4))
#print('saved to '+jsonFilePath)

In [8]:
# add empty date column to prediction_history
prediction_history['date'] = ''
result = prediction_history.to_json()
parsed = json.loads(result)
jsonFilePath='src/models/buildingMLModel/data/external/prediction_history.json'
with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(parsed, indent=4))
print('saved to '+jsonFilePath)


saved to src/models/buildingMLModel/data/external/prediction_history.json


In [9]:
prediction_history

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref,predicted fighter odds,predicted opponent odds,average bookie odds,correct?,date
0,Angela Hill,,145,145,150,144,,+150,150,+150,146,,Mackenzie Dern,,-175,-170,-190,-186,,-190,-175,-188,-171,,305,-305,"[148, -181]",,
1,Anthony Hernandez,,-210,-200,-235,-245,,-235,-225,-250,-191,,Edmen Shahbazyan,,170,170,185,186,,185,188,+200,166,,-139,139,"[-224, 181]",,
2,Emily Ducote,,130,125,128,128,,+128,130,+120,133,,Loopy Godinez,,-160,-150,-159,-164,,-159,-160,-150,-153,,210,-210,"[128, -157]",,
3,Andre Fialho,,185,190,180,172,,+180,188,+175,180,,Joaquin Buckley,,-225,-230,-230,-225,,-230,-225,-225,-210,,157,-157,"[181, -225]",,
4,Diego Ferreira,,-155,-150,-167,-156,,-167,-160,-163,-163,,Michael Johnson,,125,125,135,122,,135,130,+130,143,,-144,144,"[-160, 131]",,
5,Karolina Kowalkiewicz,,-135,-130,-137,-140,,-137,-125,-138,-130,,Vanessa Demopoulos,,110,110,110,110,,110,100,+110,110,,125,-125,"[-134, 109]",,
6,Gilbert Urbina,,100,-105,-105,102,,-105,100,+100,-105,,Orion Cosce,,-120,-115,-118,-130,,-118,-125,-125,-115,,-144,144,"[-2, -121]",,
7,Ilir Latifi,,165,170,165,152,,+165,163,+150,169,,Rodrigo Nascimento,,-200,-200,-210,-196,,-210,-188,-188,-194,,129,-129,"[162, -198]",,
8,Chase Hooper,,110,115,116,108,,+116,110,+110,102,,Nick Fiore,,-135,-135,-143,-138,,-143,-138,-138,-122,,-198,198,"[111, -136]",,
9,Natalia Silva,,-900,-900,-910,-1100,,-910,-800,-800,-1000,,Victoria Leonardo,,575,600,600,600,,600,550,+500,700,,-334,334,"[-915, 591]",,


In [ ]:
prediction_history = pd.concat([vegas_odds, prediction_history], axis = 0).reset_index(drop=True)

In [ ]:
prediction_history[['fighter name','opponent name','predicted fighter odds',	'predicted opponent odds',	'correct?',	'date']]

In [ ]:
# save prediction history to csv
prediction_history.to_csv('src/models/buildingMLModel/data/processed/prediction_history.csv', index=False)


In [ ]:
# lets clean up prediction history
# first. If two people never fought each other, we can drop the fight
# call this every time BEFORE we merge with vegas_odds
# plug in the prediction_history dataframe and ufc_fights dataframe
ufc_fights = pd.read_csv('src/models/buildingMLModel/data/external/ufc_fights.csv')
def drop_non_ufc_fights(prediction_history, ufc_fights):
    